In [1]:
ls

0_download_dataset.py*       chickenbestlover.ipynb*  __pycache__/
1_train_predictor_all.sh*    dataset/                 README.md*
1_train_predictor.py*        fig/                     requirements.txt*
2_anomaly_detection_all.sh*  LICENSE*                 result/
2_anomaly_detection.py*      model/                   save/
anomalyDetector.py*          png2gif.py*              test_dataset*
bata.csv*                    preprocess_data.py*      train_dataset*


In [2]:
import numpy as np
import pandas as pd
from multiprocessing.pool import ThreadPool
import csv
import matplotlib.pyplot as plt
import argparse
import time
import torch
import torch.nn as nn
import preprocess_data
from model import model
from torch import optim
from pathlib import Path
import pickle
from anomalyDetector import fit_norm_distribution_param
%matplotlib inline

In [3]:
base = ['vnf label', 'vnf min cpu', 'vnf min mem', 'vnf min sto', 'vnf vm cpu',
       'vnf vm mem', 'vnf vm sto', 'vnf pm id', 'vnf cpu usage',
       'vnf mem usage', 'vnf sto usage', 'vnf before', 'vnf after',
       'vnf label.1', 'vnf min cpu.1', 'vnf min mem.1', 'vnf min sto.1',
       'vnf vm cpu.1', 'vnf vm mem.1', 'vnf vm sto.1', 'vnf pm id.1',
       'vnf cpu usage.1', 'vnf mem usage.1', 'vnf sto usage.1', 'vnf before.1',
       'vnf after.1', 'sla', 'fg id', 'flow traffic', 'flow latency',
       'flow bnd usage', 'flow packet loss', 'min cpu affinity',
       'min mem affinity', 'min sto affinity', 'conflicts affinity',
       'real_affinity', 'static_affinity', 'predicted_affinity',
       'prediction_time', 'TF', 'MTT', 'MTT_upper', 'MTT_lower', 'MTTF_R',
       'MTTF_EM', 'MTTF_C', 'MTTF_TDDB', 'MTTF_SM', 'MTTFF_TC', 'A', 'AEM',
       'AC', 'ATDDB', 'ASM', 'ATC', 'TAA', 'QRED', 'QR', 'PUE', 'DCie', 'cost',
       'TIMESTAMP', 'EXTERNAL_TEMP', 'ROOM_TEMP', 'MTTF_IC', 'A_TC', 'Q_DIT',
       'TPF', 'AIRFLOW', 'TAAF', 'DeltaT_de', 'QD']
       
       
#Select your global variable among one of above, and place it after base below


base = 'vnf cpu usage'


In [4]:
fgs = pd.read_csv('../fgs.csv')

In [5]:
fgs.head()

,,,,4963594775,1,2
proxy.9.5698,traffic counter.7.5664,0.971442,82.243099,1.794354,9.341767,100.0
5402488769,5,1.000000,NaN,NaN,NaN,NaN
traffic counter.7.7413,proxy.9.11244,4.060322,119.086081,16.548350,8.474562,150.0
proxy.9.11244,proxy.9.2362,0.268318,23.865256,11.573841,9.248256,150.0
proxy.9.2362,traffic counter.7.9524,2.961259,80.083566,18.280812,4.783467,150.0


In [6]:
vns = pd.read_csv('../vnfs.csv', header=None)

In [7]:
vnf_id = vns[vns.columns[0]]
vnf_type = vns[vns.columns[1]]
vnf_scheduling = vns[vns.columns[2]]
vnf_pm = vns[vns.columns[3]]
vnf_fg = vns[vns.columns[4]]
flavor_data = vns[vns.columns[5:8]]
vm_data = vns[vns.columns[8:11]]
usage_data = vns[vns.columns[11:15]]

In [8]:
output = pd.read_csv('../new_results_out.csv', sep =';')#header=None)
output.drop('Unnamed: 0',axis = 1, inplace=True)

In [9]:
output.head()

,vnf label,vnf min cpu,vnf min mem,vnf min sto,vnf vm cpu,vnf vm mem,vnf vm sto,vnf pm id,vnf cpu usage,vnf mem usage,...,EXTERNAL_TEMP,ROOM_TEMP,MTTF_IC,A_TC,Q_DIT,TPF,AIRFLOW,TAAF,DeltaT_de,QD
0,dpi.2.136144,0.035914,0.226393,0.001757,0.06250,0.12720,0.001930,257345416,71.88000,1.75780,...,15,23,2435.249666,0.996726,1.839678,14.999377,0.131858,0.872117,6.132259,6.132259
1,traffic counter.7.40368,0.063818,0.017674,0.000223,0.06189,0.03821,0.000386,3488796635,3.51600,6.34800,...,15,24,49.077683,0.859840,1.801941,14.999402,0.142946,0.768427,6.006469,6.006469
2,proxy.9.13456,0.018672,0.020912,0.000006,0.01250,0.01590,0.000404,4820294844,0.18044,0.13352,...,55,25,82.372536,0.911478,1.800100,54.999420,0.152569,0.757640,6.000332,6.000332
3,firewall.3.36933,0.140469,0.021706,0.000068,0.09375,0.03198,0.000070,3550430599,26.46000,6.44600,...,40,23,0.044220,0.005497,1.814606,39.999481,0.132889,0.807089,6.048686,6.048686
4,dpi.2.92594,0.015708,0.050654,0.000810,0.06250,0.08569,0.000926,4820072706,41.74000,14.42800,...,21,29,0.011200,0.001398,1.823040,20.999427,0.189031,0.790021,6.076802,6.076802


In [10]:
output.columns

Index(['vnf label', 'vnf min cpu', 'vnf min mem', 'vnf min sto', 'vnf vm cpu',
       'vnf vm mem', 'vnf vm sto', 'vnf pm id', 'vnf cpu usage',
       'vnf mem usage', 'vnf sto usage', 'vnf before', 'vnf after',
       'vnf label.1', 'vnf min cpu.1', 'vnf min mem.1', 'vnf min sto.1',
       'vnf vm cpu.1', 'vnf vm mem.1', 'vnf vm sto.1', 'vnf pm id.1',
       'vnf cpu usage.1', 'vnf mem usage.1', 'vnf sto usage.1', 'vnf before.1',
       'vnf after.1', 'sla', 'fg id', 'flow traffic', 'flow latency',
       'flow bnd usage', 'flow packet loss', 'min cpu affinity',
       'min mem affinity', 'min sto affinity', 'conflicts affinity',
       'real_affinity', 'static_affinity', 'predicted_affinity',
       'prediction_time', 'TF', 'MTT', 'MTT_upper', 'MTT_lower', 'MTTF_R',
       'MTTF_EM', 'MTTF_C', 'MTTF_TDDB', 'MTTF_SM', 'MTTFF_TC', 'A', 'AEM',
       'AC', 'ATDDB', 'ASM', 'ATC', 'TAA', 'QRED', 'QR', 'PUE', 'DCie', 'cost',
       'TIMESTAMP', 'EXTERNAL_TEMP', 'ROOM_TEMP', 'MTTF_IC', 'A_T

In [13]:
output[base].shape

(195379,)

In [14]:
output = output[:12322]
output = output[base]

In [15]:
dataset = np.stack((output, vnf_id, np.zeros(len(vnf_id))), axis = 1).tolist()

In [16]:
dataset[0]

[71.88, 126.0, 0.0]

In [19]:
with open('train_dataset','wb') as pkl:
    pickle.dump(dataset[:12000], pkl)
    
with open('test_dataset','wb') as pkl:
    pickle.dump(dataset[12000:], pkl)